# Understanding primitive inputs and outputs

This page gives an overview of the inputs and outputs of the V2 primitives implemented by Qiskit Runtime which will execute workloads on an IBM Quantum backend. These interfaces provide you with the ability to efficiently define vectorized workloads by using a data structure known as a Primitive Unified Bloc (PUB). These PUBs are used as the fundamental unit of work which a backend needs in order to execute these workloads. 

Once these workloads are submitted as jobs to the IBM Quantum platform, they are executed and the data returned is dependent on the PUBs which were executed, as well as any error mitigation or suppression options that were defined when the primitive was invoked.

## Overview of PUBS

The PUB data structure defines individual workloads to be submitted to a backend for execution on a QPU. It is a tuple which contains four values:
- A single `QuantumCircuit` which possibly containing one or more `Parameters`
- A list of one or more observables, which specify the expectation values to estimate. The data can be in any one of the `ObservablesArrayLike` format such as `Pauli`, `SparsePauliOp`, or `str`. (Not needed for the `Sampler` primitive)
- A collection of parameter values to bind the circuit against $\theta_k$
- (Optionally) a target precision for expectation values to estimate (Not needed for the `Sampler` primitive)

When invoking a primitive's [`run()`](api/qiskit-ibm-runtime/qiskit_ibm_runtime.EstimatorV2#qiskit_ibm_runtime.EstimatorV2.run) method, the argument expected is a `list` of one or PUBs, the data of which will depend on the computation being submitted to the backend. 

The below code demonstrates an example set of vectorized inputs to the `Estimator` primitive and executes them on an IBM Quantum backend as a single `RuntimeJobV2 ` object.

<Admonition type="caution" title="Important">
To ensure faster and more efficient results, as of 1 March 2024, circuits and observables need to be transformed to only use instructions supported by the system (referred to as *instruction set architecture (ISA)* circuits and observables) before being submitted to the Qiskit Runtime primitives.  See the [transpilation documentation](../transpile) for instructions to transform circuits.  Due to this change, the primitives will no longer perform layout or routing operations.  Consequently, transpilation options referring to those tasks will no longer have any effect. By default, all primitives except Sampler V2 still optimize the input circuits. To bypass all optimization, set `optimization_level=0`.

*Exception*: When you initialize the Qiskit Runtime Service with the Q-CTRL channel strategy (example below), abstract circuits are still supported.

``` python
service = QiskitRuntimeService(channel="ibm_cloud", channel_strategy="q-ctrl")
```

</Admonition>

In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit_ibm_runtime import (
    EstimatorV2 as Estimator,
    QiskitRuntimeService,
)
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit.quantum_info import Pauli, SparsePauliOp
import numpy as np

# Instantiate runtime service and get
# the least busy backend
service = QiskitRuntimeService()
backend = service.least_busy(operational=True, simulator=False)

# Define a circuit with two parameters.
circuit = QuantumCircuit(2)
circuit.h(0)
circuit.cx(0, 1)
circuit.ry(Parameter("a"), 0)
circuit.rz(Parameter("b"), 0)
circuit.cx(0, 1)
circuit.h(0)

# Transpile the circuit
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
transpiled_circuit = pm.run(circuit)
layout = transpiled_circuit.layout


# Now define a sweep over parameter values, where the two axes are over the
# two parameters in the circuit.
params = np.vstack(
    [
        np.linspace(-np.pi, np.pi, 100),
        np.linspace(-4 * np.pi, 4 * np.pi, 100),
    ]
).T

# Define three observables. The inner length-1 lists cause this array of
# observables to have shape (3, 1), rather than shape (3,) if they were
# omitted.
observables = [
    [SparsePauliOp(["XX", "IY"], [0.5, 0.5])],
    [SparsePauliOp("XX")],
    [SparsePauliOp("IY")],
]
# Apply the same layout as the transpiled circuit.
observables = [
    [observable.apply_layout(layout) for observable in observable_set]
    for observable_set in observables
]

# Estimate the expectation value for all 300 combinations of observables
# and parameter values, where the pub result will have shape (3, 100).
#
# This shape is due to our array of parameter bindings having shape
# (100, 2), combined with our array of observables having shape (3, 1).
pub = (transpiled_circuit, observables, params)

# Instantiate the new estimator object, then run the transpiled circuit
# using the set of parameters and observables.
estimator = Estimator(mode=backend)
job = estimator.run([pub])
result = job.result()

# Print out the results and job metadata
print(f" > Expectation value: {result[0].data.evs}")
print(f" > Metadata: {result[0].metadata}")

## Overview of execution results

Once one or more PUBs are sent to a QPU for execution and have successfully completed their associated jobs, the data is returned as a `PrimitiveResult` object.